In [0]:
dbutils.widgets.text(name="env", defaultValue="", label="Enter environment")
env = dbutils.widgets.get("env")
env

'dev'

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql import Window


In [0]:
%run "./paths"

('abfss://landing@dlsunitycat.dfs.core.windows.net/',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/bronze',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/silver',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/gold')

In [0]:
catalog = f'{env}_catalog'
table = 'rates'
rates_table = f'{catalog}{bronze_tables[table]}'
rates_table

'dev_catalog.bronze.rates_bronze'

In [0]:
rates = spark.table(rates_table)

In [0]:
@udf
def convert_ages(age):
    if age < 20:
        return "0 - 19"
    elif age < 30:
        return "20 - 29"
    elif age < 40:
        return "30 - 39"
    elif age < 50:
        return "40 - 49"
    elif age < 60:
        return "50 - 59"
    return "60+"

case_age = (F.when(rates.age == "0-20", "19")
            .when(rates.age == "64 and over", "64")
            .when(rates.age == "65 and over", "65")
            .when(rates.age == "0-14", "10")
            .otherwise(rates.age))

rates_1 = (
    rates.drop("date_ingested")
    .dropDuplicates()
    .filter(F.col("age") != "Family Option")
    .withColumn("age", case_age)
    .withColumn("age", F.col("age").cast(IntegerType()))
    .withColumn("age_category", convert_ages("age"))
    .withColumn("date_ingested", F.current_timestamp())
)

window_spec = Window.partitionBy("plan_id")

rates_filtered = rates_1.withColumn("has_zero", F.max(F.when(F.col("rate") == 0, 1).otherwise(0)).over(window_spec)) \
                        .filter(F.col("has_zero") == 0) \
                        .drop("has_zero")

rates_filtered.write.format("delta").mode("overwrite").save(silver_paths[table])

